<h2 style="color:brown;">Install Dependencies</h2>

In [1]:
!pip install albumentations opencv-python



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


<h2 style="color:brown;">Import dependencies</h2>

In [2]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

d:\Thesis\Code\isl-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


<h2 style="color:brown;">Augmentation Functions process</h2>

In [17]:
# Define the augmentation 
def get_transform():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=0.5),
        A.Blur(blur_limit=3, p=0.2),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [18]:
def augment_image(image, bboxes, class_labels, num_augments=29):
    augmented_images = []
    augmented_bboxes = []
    augmented_class_labels = []

    transform = get_transform()

    for _ in range(num_augments):
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_images.append(augmented['image'])
        augmented_bboxes.append(augmented['bboxes'])
        augmented_class_labels.append(augmented['class_labels'])

    return augmented_images, augmented_bboxes, augmented_class_labels

<h2 style="color:brown;">Define path to save images</h2>

In [19]:
# Paths
input_folder = 'D:/Thesis/Code/Final-data/images/train'
output_images_folder = 'D:/Thesis/Code/pre-processing/final-img/train1'
output_labels_folder = 'D:/Thesis/Code/pre-processing/final-img/train2'
annotation_folder = 'D:/Thesis/Code/Final-data/labels/train'  # Assuming YOLO format annotations

In [20]:
if not os.path.exists(output_images_folder):
    os.makedirs(output_images_folder)

if not os.path.exists(output_labels_folder):
    os.makedirs(output_labels_folder)

<h2 style="color:brown;">Processing Augmentation</h2>

In [21]:
# Process each image in the input folder
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        image_height, image_width = image.shape[:2]

        # Load corresponding annotation
        annotation_path = os.path.join(annotation_folder, os.path.splitext(filename)[0] + '.txt')
        bboxes = []
        class_labels = []

        if os.path.exists(annotation_path):
            with open(annotation_path, 'r') as file:
                for line in file.readlines():
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)

        # Save the original image
        original_image_path = os.path.join(output_images_folder, filename)
        cv2.imwrite(original_image_path, image)

        # Save the original annotation
        original_annotation_path = os.path.join(output_labels_folder, os.path.splitext(filename)[0] + '.txt')
        with open(original_annotation_path, 'w') as file:
            for bbox, class_id in zip(bboxes, class_labels):
                file.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

        # Generate and save augmented images (29 augmentations per image)
        augmented_images, augmented_bboxes, augmented_class_labels = augment_image(image, bboxes, class_labels, num_augments=29)
        
        for i, (aug_img, aug_bboxes, aug_labels) in enumerate(zip(augmented_images, augmented_bboxes, augmented_class_labels)):
            augmented_image_filename = f"{os.path.splitext(filename)[0]}_aug_{i}.jpg"
            augmented_image_path = os.path.join(output_images_folder, augmented_image_filename)
            augmented_annotation_filename = f"{os.path.splitext(filename)[0]}_aug_{i}.txt"
            augmented_annotation_path = os.path.join(output_labels_folder, augmented_annotation_filename)

            # Save augmented image
            aug_img_np = aug_img.permute(1, 2, 0).numpy()[:, :, ::-1]  # Convert CHW to HWC and RGB to BGR
            cv2.imwrite(augmented_image_path, aug_img_np)

            # Save augmented annotation
            with open(augmented_annotation_path, 'w') as file:
                for bbox, class_id in zip(aug_bboxes, aug_labels):
                    file.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

print('Augmentation complete.')

100%|██████████| 481/481 [29:59<00:00,  3.74s/it]

Augmentation complete.


<h2 style="color:brown;">Repeat the process of Augmentation for Validation</h2>

In [22]:
# Paths
input_folder = 'D:/Thesis/Code/Final-data/images/val'
output_images_folder = 'D:/Thesis/Code/pre-processing/final-img/val1'
output_labels_folder = 'D:/Thesis/Code/pre-processing/final-img/val2'
annotation_folder = 'D:/Thesis/Code/Final-data/labels/val'  # Assuming YOLO format annotations

In [23]:
if not os.path.exists(output_images_folder):
    os.makedirs(output_images_folder)

if not os.path.exists(output_labels_folder):
    os.makedirs(output_labels_folder)

In [24]:
# Process each image in the input folder
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        image_height, image_width = image.shape[:2]

        # Load corresponding annotation
        annotation_path = os.path.join(annotation_folder, os.path.splitext(filename)[0] + '.txt')
        bboxes = []
        class_labels = []

        if os.path.exists(annotation_path):
            with open(annotation_path, 'r') as file:
                for line in file.readlines():
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)

        # Save the original image
        original_image_path = os.path.join(output_images_folder, filename)
        cv2.imwrite(original_image_path, image)

        # Save the original annotation
        original_annotation_path = os.path.join(output_labels_folder, os.path.splitext(filename)[0] + '.txt')
        with open(original_annotation_path, 'w') as file:
            for bbox, class_id in zip(bboxes, class_labels):
                file.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

        # Generate and save augmented images (29 augmentations per image)
        augmented_images, augmented_bboxes, augmented_class_labels = augment_image(image, bboxes, class_labels, num_augments=29)
        
        for i, (aug_img, aug_bboxes, aug_labels) in enumerate(zip(augmented_images, augmented_bboxes, augmented_class_labels)):
            augmented_image_filename = f"{os.path.splitext(filename)[0]}_aug_{i}.jpg"
            augmented_image_path = os.path.join(output_images_folder, augmented_image_filename)
            augmented_annotation_filename = f"{os.path.splitext(filename)[0]}_aug_{i}.txt"
            augmented_annotation_path = os.path.join(output_labels_folder, augmented_annotation_filename)

            # Save augmented image
            aug_img_np = aug_img.permute(1, 2, 0).numpy()[:, :, ::-1]  # Convert CHW to HWC and RGB to BGR
            cv2.imwrite(augmented_image_path, aug_img_np)

            # Save augmented annotation
            with open(augmented_annotation_path, 'w') as file:
                for bbox, class_id in zip(aug_bboxes, aug_labels):
                    file.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

print('Augmentation complete.')

100%|██████████| 90/90 [07:58<00:00,  5.31s/it]

Augmentation complete.
